In [2]:
!pip3 install pyrealsense2 open3d

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 15.6 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.8 MB/s eta 0:00:00 MB/s eta 0:00:0101mm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.5/746.5 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 87.2 MB/s eta 0:00:000m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 36.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import open3d as o3d

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
 
# Enable depth and color streams
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Get depth scale for converting depth values to meters
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# Intrinsic parameters
intrinsics = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intrinsics.fx, intrinsics.fy, intrinsics.ppx, intrinsics.ppy

def depth_to_3d(x, y, depth):
    """ Convert pixel (x, y) and depth to 3D world coordinates. """
    z = depth * depth_scale
    if z == 0:  # Ignore invalid depth points
        return None
    X = (x - cx) * z / fx
    Y = (y - cy) * z / fy
    return np.array([X, Y, z])

def detect_ball(color_image):
    """ Detect a ball in the color image using color thresholding. """
    hsv = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)
    
    # Define color range for the ball (adjust as needed)
    lower_red = np.array([0, 140, 180])
    upper_red = np.array([20, 230, 255])
    
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        if radius > 5:  # Filter out small noise
            return int(x), int(y), int(radius)
    return None

try:
    while True:
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert frames to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Detect the ball
        ball = detect_ball(color_image)
        if ball:
            x, y, radius = ball

            # Get depth value at the detected position
            depth_value = depth_image[y, x]
            ball_position = depth_to_3d(x, y, depth_value)

            if ball_position is not None:
                print(f"Ball detected at: {ball_position}")

                # Draw the detected ball on the color image
                cv2.circle(color_image, (x, y), radius, (0, 255, 0), 2)
                cv2.putText(color_image, f"3D: {ball_position[0]:.2f}, {ball_position[1]:.2f}, {ball_position[2]:.2f}", 
                            (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Show images
        cv2.imshow("Color Image", color_image)
        # cv2.imshow("Depth Image", cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET))

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/sakib/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Ball detected at: [ 0.45717211 -0.28436486  0.73600003]
Ball detected at: [ 0.44907887 -0.2805012   0.72600003]
Ball detected at: [ 0.44781318 -0.28088757  0.72700003]
Ball detected at: [ 0.44776906 -0.2839387   0.73000003]
Ball detected at: [ 0.43364577 -0.27200117  0.70400003]
Ball detected at: [ 0.44842915 -0.28127393  0.72800003]
Ball detected at: [ 0.44226941 -0.27741028  0.71800003]
Ball detected at: [ 0.44042149 -0.27810434  0.71500003]
Ball detected at: [ 0.43795495 -0.27586482  0.71400003]
Ball detected at: [ 0.44776906 -0.28204666  0.73000003]
Ball detected at: [ 0.43857356 -0.27693747  0.71200003]
Ball detected at: [ 0.44165343 -0.27702391  0.71700003]
Ball detected at: [ 0.43734162 -0.27431936  0.71000003]
Ball detected at: [ 0.44776906 -0.28204666  0.73000003]
Ball detected at: [ 0.4471972  -0.28238287  0.72600003]
Ball detected at: [ 0.43979509 -0.27888225  0.71700003]
Ball detected at: [ 0.44224862 -0.27856938  0.72100003]
Ball detected at: [ 0.45022259 -0.28549453  0.73

In [13]:
import pyrealsense2 as rs
import cv2
import numpy as np

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

def get_hsv_value(event, x, y, flags, param):
    """Callback function to get HSV values on mouse click."""
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse click
        hsv_pixel = hsv[y, x]  # Get HSV value at clicked position
        print(f"HSV at ({x}, {y}): {hsv_pixel}")

try:
    while True:
        # Wait for a new frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue
        
        # Convert frame to numpy array
        color_image = np.asanyarray(color_frame.get_data())
        hsv = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)  # Convert to HSV
        
        # Display image
        cv2.imshow("RealSense Camera", color_image)
        cv2.setMouseCallback("RealSense Camera", get_hsv_value)  # Set mouse click event

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop the RealSense pipeline
    pipeline.stop()
    cv2.destroyAllWindows()


HSV at (445, 220): [103 142 120]


In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()

# Enable depth and color streams
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Get depth scale for converting depth values to meters
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# Get camera intrinsics
intrinsics = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intrinsics.fx, intrinsics.fy, intrinsics.ppx, intrinsics.ppy

# Function to convert depth to 3D world coordinates
def depth_to_3d(x, y, depth):
    z = depth * depth_scale
    if z == 0:  # Ignore invalid depth points
        return None
    X = (x - cx) * z / fx
    Y = (y - cy) * z / fy
    return np.array([X, Y, z])

# Initialize previous frame for motion detection
previous_gray = None

try:
    while True:
        # Capture frames
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Convert to grayscale
        gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # If first frame, store and continue
        if previous_gray is None:
            previous_gray = gray
            continue

        # Compute absolute difference between current and previous frame
        frame_diff = cv2.absdiff(previous_gray, gray)

        # Apply threshold to highlight differences
        _, thresh = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)

        # Find contours of the moving regions
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            (x, y), radius = cv2.minEnclosingCircle(largest_contour)
            
            if radius > 10:  # Filter out small noise
                x, y, radius = int(x), int(y), int(radius)

                # Get depth value at detected position
                depth_value = depth_image[y, x]
                ball_position = depth_to_3d(x, y, depth_value)

                if ball_position is not None:
                    print(f"Moving object detected at: {ball_position}")

                    # Draw detection circle
                    cv2.circle(color_image, (x, y), radius, (0, 255, 0), 2)
                    cv2.putText(color_image, f"3D: {ball_position[0]:.2f}, {ball_position[1]:.2f}, {ball_position[2]:.2f}", 
                                (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Update previous frame
        previous_gray = gray.copy()

        # Display results
        cv2.imshow("Color Image", color_image)
        cv2.imshow("Motion Mask", thresh)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


Moving object detected at: [-2.16034805  0.67731169  4.2890002 ]
Moving object detected at: [1.97886493 0.22913353 2.53100012]
Moving object detected at: [-0.5409075   0.35128289  0.83700004]
Moving object detected at: [-2.08863534  0.67321392  4.06300019]
Moving object detected at: [-2.1948321   0.71438666  4.38000021]
Moving object detected at: [-0.61581239  0.28216642  1.73000008]
Moving object detected at: [-2.7749496   0.95954493  5.37100026]
Moving object detected at: [-2.87826849  1.00949917  5.06300024]
Moving object detected at: [-2.80493947  0.92938549  4.1730002 ]
Moving object detected at: [ 1.02303893 -1.47690531  2.99800014]
Moving object detected at: [-1.13887053 -1.16051144  2.23800011]
Moving object detected at: [-1.42956795 -1.15636306  2.23000011]
Moving object detected at: [ 2.25070666 -2.00468692  4.2020002 ]
Moving object detected at: [-1.15912944 -1.20747888  2.41300011]
Moving object detected at: [-1.42001473 -1.11409642  2.34800011]
Moving object detected at: [